# Smart Cookies - Location Exposure Extract
#### based on - C3.ai COVID-19 Data Lake Quickstart in Python
#### Location Exposure doco: https://c3.ai/covid-19-api-documentation/#tag/LocationExposure

Loading Dependencies Below:

In [21]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np

Ensuring that the most recent version of pandas (>= 1.0.0)

In [22]:
print("pandas version", pd.__version__)
assert pd.__version__[0] >= "1", "To use this notebook, upgrade to the newest version of pandas. See https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html for details."

pandas version 1.1.3


<a id="helpers"></a>
## Helper methods for accessing the API

The helper methods in `c3aidatalake.py` convert a JSON response from the C3.ai APIs to a Pandas DataFrame. You may wish to view the code in `c3aidatalake.py` before running the code below. 

`c3aidatalake.py` is a API file provided by C3.ai

In [23]:
import c3aidatalake 

### Depends on output file from Daphne's notebook: outbreaklocation_by_county
Note: For testing, limited to Texas only and the first 100 counties.

In [24]:
# get the location exposure data for a pair of counties, return it as a df
def eachlocationExposures(locationTarget , locationVisited):

    exposure = c3aidatalake.read_data_json(
        "locationexposure",
        "getlocationexposures",
            {
                "spec":
                {
                    "locationTarget": locationTarget,
                    "locationVisited": locationVisited,
                    "start": "2020-09-01",
                    "end": "2020-10-31"
                }
            }
        )

    lex_df = pd.json_normalize(exposure["locationExposures"]["value"])
    print ('Fetched location exposures for:' + locationTarget + ' - ' + locationVisited + '. Rows returned: ' + str(len(lex_df)))
    
    if len(lex_df) > 0:
        device_counts_df = pd.json_normalize(exposure["deviceCounts"]["value"])
        eachlocationExposures_df = pd.merge(lex_df, device_counts_df, how='left', left_on=['locationTarget','timestamp'], right_on=['locationTarget','timestamp'])
    else:
        eachlocationExposures_df= lex_df

    eachlocationExposures_df = eachlocationExposures_df.rename({'value_x':'value_Location_exposure_index', 'value_y':'value_Device_count'}, axis='columns')
    # print(eachlocationExposures_df)
    
    return eachlocationExposures_df


In [26]:
counties_df = pd.read_csv('outbreaklocation_by_county.csv')

counties_df = counties_df[counties_df['id'].str.contains("_Texas_")]

counties_df = counties_df.head(5)

all_lex = []

for each_id_Target in counties_df['id']:
    for each_id_Visited in counties_df['id']:
        if each_id_Target != each_id_Visited:
            lex = eachlocationExposures(each_id_Visited, each_id_Target)
            all_lex.append(lex)

all_lex_df = pd.concat(all_lex, ignore_index=True)
print(all_lex_df)
all_lex_df.to_csv('locationExposures.csv')
print ('Finished! Wrote output file with ' + str(len(all_lex_df)) + 'rows')


Fetched location exposures for:Andrews_Texas_UnitedStates - Anderson_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Angelina_Texas_UnitedStates - Anderson_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Aransas_Texas_UnitedStates - Anderson_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Archer_Texas_UnitedStates - Anderson_Texas_UnitedStates. Rows returned: 0
Fetched location exposures for:Anderson_Texas_UnitedStates - Andrews_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Angelina_Texas_UnitedStates - Andrews_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Aransas_Texas_UnitedStates - Andrews_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Archer_Texas_UnitedStates - Andrews_Texas_UnitedStates. Rows returned: 0
Fetched location exposures for:Anderson_Texas_UnitedStates - Angelina_Texas_UnitedStates. Rows returned: 17
Fetched location exposures for:Andrews_Te